In [106]:
!pip3 install --quiet matplotlib numpy pandas scikit-learn seaborn

# Data Cleaning

In [107]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [108]:
# Changing the path to the root of the repository

if not os.path.exists('data'):
    current_path = os.getcwd()
    root_path = os.path.abspath(os.path.join(current_path, '..'))
    os.chdir(root_path)
    print(f'Changed working directory to: {os.getcwd()}')

In [109]:
# Load raw data
acorn_details = pd.read_csv('data/00_raw/acorn_details.csv', encoding='ISO-8859-1')
temperatures = pd.read_csv('data/00_raw/temperatures.csv', sep=';', decimal=',', encoding='utf-8')
uk_bank_holidays = pd.read_csv('data/00_raw/uk_bank_holidays.csv')
weather_daily = pd.read_csv('data/00_raw/weather_daily_darksky.csv')
weather_hourly = pd.read_csv('data/00_raw/weather_hourly_darksky.csv')

# Load processed data from parquet
group_4_daily_predict = pd.read_parquet('data/02_processed/parquet/group_4_daily_predict.parquet')
group_4_half_hourly_predict = pd.read_parquet('data/02_processed/parquet/group_4_half_hourly_predict.parquet')
group_4_daily = pd.read_parquet('data/02_processed/parquet/group_4_daily.parquet')
group_4_half_hourly = pd.read_parquet('data/02_processed/parquet/group_4_half_hourly.parquet')

In [110]:
# Fixing Datetime formats
temperatures['DateTime'] = pd.to_datetime(temperatures['DateTime'], format='mixed')
uk_bank_holidays['Bank holidays'] = pd.to_datetime(uk_bank_holidays['Bank holidays'], format='mixed')

# Convert all datetime-like columns in weather_daily to datetime format
datetime_columns = ['temperatureMaxTime', 'temperatureMinTime', 'apparentTemperatureMinTime', 
                   'apparentTemperatureHighTime', 'time', 'sunsetTime', 'sunriseTime', 
                   'temperatureHighTime', 'uvIndexTime', 'temperatureLowTime', 
                   'apparentTemperatureMaxTime', 'apparentTemperatureLowTime']

for col in datetime_columns:
    weather_daily[col] = pd.to_datetime(weather_daily[col])

# Data Extraction


In [111]:
df_weather_hourly = pd.read_csv('data/00_raw/weather_hourly_darksky.csv')


In [112]:
df_weather_hourly.head()

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy
2,3.70,98,9.46,2011-11-11 02:00:00,8.79,1016.36,7.76,3.17,rain,partly-cloudy-night,0.96,Partly Cloudy
3,3.12,99,9.23,2011-11-11 03:00:00,8.63,1016.28,7.44,3.25,rain,fog,0.96,Foggy
4,1.85,111,9.26,2011-11-11 04:00:00,9.21,1015.98,7.24,3.70,rain,fog,1.00,Foggy


# Preprocessing


In [114]:
df_weather_hourly = df_weather_hourly.sort_values(by='time', ascending=True).reset_index(drop=True)

df_weather_hourly.isna().sum()

visibility              0
windBearing             0
temperature             0
time                    0
dewPoint                0
pressure               13
apparentTemperature     0
windSpeed               0
precipType              0
icon                    0
humidity                0
summary                 0
dtype: int64

In [115]:
nan_rows = df_weather_hourly[df_weather_hourly.isna().any(axis=1)]
nan_rows

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
4530,11.27,189,12.99,2012-05-07 18:00:00,8.29,NaN,12.99,4.22,rain,partly-cloudy-day,0.73,Partly Cloudy
8655,11.27,21,7.29,2012-10-26 15:00:00,3.68,NaN,4.21,4.99,rain,partly-cloudy-day,0.78,Mostly Cloudy
10303,9.66,250,9.65,2013-01-03 07:00:00,8.67,NaN,7.07,5.21,rain,partly-cloudy-night,0.94,Mostly Cloudy
14077,9.98,48,13.42,2013-06-09 13:00:00,7.76,NaN,13.42,5.07,rain,clear-day,0.69,Clear
15336,9.98,193,19.16,2013-08-01 00:00:00,16.78,NaN,19.38,2.80,rain,clear-night,0.86,Clear
15697,9.98,217,17.01,2013-08-16 01:00:00,14.84,NaN,17.04,3.83,rain,clear-night,0.87,Clear
15935,6.39,10,17.56,2013-08-25 23:00:00,15.49,NaN,17.66,3.75,rain,clear-night,0.88,Clear
15936,4.39,358,17.02,2013-08-26 00:00:00,15.42,NaN,17.13,2.72,rain,clear-night,0.90,Clear
16273,9.51,221,9.39,2013-09-09 01:00:00,8.03,NaN,8.03,2.61,rain,clear-night,0.91,Clear
16276,9.46,224,9.99,2013-09-09 04:00:00,7.72,NaN,9.64,1.51,rain,clear-night,0.86,Clear


In [116]:
#plt.figure(figsize=(14, 5))
#plt.plot(df_weather_hourly['time'], df_weather_hourly['pressure'])
#plt.xlabel('Time')
#plt.ylabel('Pressure')
#plt.title('Pressure Over Time')
#plt.tight_layout()
#plt.show()

# Function to get previous, current, and next rows for NaN values in a specified column
def get_prev_next_rows(df, column):
    # Identify indices where the specified column is NaN
    nan_rows = df[df[column].isna()]
    rows = []
    for idx in nan_rows.index:
        loc = df.index.get_loc(idx)
        # previous row by position
        if loc > 0:
            rows.append(df.iloc[loc - 1])
        # current NaN row
        rows.append(df.iloc[loc])
        # next row by position
        if loc < len(df.index) - 1:
            rows.append(df.iloc[loc + 1])
    prev_next_df = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)
    return prev_next_df

#get_prev_next_rows(df_weather_hourly, 'pressure')

In [ ]:
# Interpolate NaN values in the 'pressure' column using linear interpolation (previous and next points)
df_weather_hourly['pressure'] = df_weather_hourly['pressure'].interpolate(method='linear', limit_direction='both')
df_weather_hourly.loc[nan_indices, ['time', 'pressure']], df_weather_hourly.isna().sum()

(                      time     pressure
 4530   2012-05-07 18:00:00  1011.545000
 8655   2012-10-26 15:00:00  1010.865000
 10303  2013-01-03 07:00:00  1030.525000
 14077  2013-06-09 13:00:00  1016.430000
 15336  2013-08-01 00:00:00  1014.155000
 15697  2013-08-16 01:00:00  1015.860000
 15935  2013-08-25 23:00:00  1017.596667
 15936  2013-08-26 00:00:00  1017.753333
 16273  2013-09-09 01:00:00  1017.755000
 16276  2013-09-09 04:00:00  1017.760000
 16377  2013-09-13 11:00:00  1018.195000
 20946  2014-03-22 20:00:00  1000.770000
 20947  2014-03-22 21:00:00  1001.260000,
 visibility             0
 windBearing            0
 temperature            0
 time                   0
 dewPoint               0
 pressure               0
 apparentTemperature    0
 windSpeed              0
 precipType             0
 icon                   0
 humidity               0
 summary                0
 dtype: int64)

In [118]:
# Convert the 'time' column to datetime format for easier manipulation
df_weather_hourly['time'] = pd.to_datetime(df_weather_hourly['time'])

df_weather_hourly.head()

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,13.63,160,13.49,2011-11-01 00:00:00,11.48,1008.14,13.49,3.11,rain,clear-night,0.88,Clear
1,13.26,154,12.73,2011-11-01 01:00:00,11.58,1007.88,12.73,3.08,rain,partly-cloudy-night,0.93,Partly Cloudy
2,12.94,161,13.65,2011-11-01 02:00:00,12.14,1007.09,13.65,3.71,rain,clear-night,0.91,Clear
3,12.99,170,14.13,2011-11-01 03:00:00,12.24,1006.50,14.13,3.95,rain,partly-cloudy-night,0.88,Partly Cloudy
4,12.92,180,14.17,2011-11-01 04:00:00,12.59,1006.14,14.17,3.97,rain,partly-cloudy-night,0.90,Partly Cloudy


In [119]:
# Separate quantitative (numerical) and qualitative (categorical/object) columns in df_weather_hourly
quantitative_cols = df_weather_hourly.select_dtypes(include=['number']).columns.tolist()
qualitative_cols = df_weather_hourly.select_dtypes(exclude=['number']).columns.tolist()

print("Quantitative columns:", quantitative_cols)
print("Qualitative columns:", qualitative_cols)

unique_counts = df_weather_hourly.nunique(dropna=False)

for col, count in unique_counts.items():
    print(f"{col}: {count}")


Quantitative columns: ['visibility', 'windBearing', 'temperature', 'dewPoint', 'pressure', 'apparentTemperature', 'windSpeed', 'humidity']
Qualitative columns: ['time', 'precipType', 'icon', 'summary']
visibility: 953
windBearing: 360
temperature: 2803
time: 21165
dewPoint: 2398
pressure: 4996
apparentTemperature: 3124
windSpeed: 1095
precipType: 2
icon: 7
humidity: 78
summary: 13


In [120]:
df_weather_hourly = df_weather_hourly.drop(columns=['summary'])
df_weather_hourly.head()

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity
0,13.63,160,13.49,2011-11-01 00:00:00,11.48,1008.14,13.49,3.11,rain,clear-night,0.88
1,13.26,154,12.73,2011-11-01 01:00:00,11.58,1007.88,12.73,3.08,rain,partly-cloudy-night,0.93
2,12.94,161,13.65,2011-11-01 02:00:00,12.14,1007.09,13.65,3.71,rain,clear-night,0.91
3,12.99,170,14.13,2011-11-01 03:00:00,12.24,1006.50,14.13,3.95,rain,partly-cloudy-night,0.88
4,12.92,180,14.17,2011-11-01 04:00:00,12.59,1006.14,14.17,3.97,rain,partly-cloudy-night,0.90


In [121]:
df_weather_hourly = df_weather_hourly.set_index('time')


df_weather_hourly.head()

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity
time,,,,,,,,,,
2011-11-01 00:00:00,13.63,160,13.49,11.48,1008.14,13.49,3.11,rain,clear-night,0.88
2011-11-01 01:00:00,13.26,154,12.73,11.58,1007.88,12.73,3.08,rain,partly-cloudy-night,0.93
2011-11-01 02:00:00,12.94,161,13.65,12.14,1007.09,13.65,3.71,rain,clear-night,0.91
2011-11-01 03:00:00,12.99,170,14.13,12.24,1006.50,14.13,3.95,rain,partly-cloudy-night,0.88
2011-11-01 04:00:00,12.92,180,14.17,12.59,1006.14,14.17,3.97,rain,partly-cloudy-night,0.90


In [124]:
df_weather_hourly.precipType = pd.Categorical(df_weather_hourly.precipType)
df_weather_hourly.icon = pd.Categorical(df_weather_hourly.icon)


In [133]:
# Identify quantitative columns (numeric types)
quantitative_cols = df_weather_hourly.select_dtypes(include=['number']).columns.tolist()

# Create a half-hourly index, then interpolate numeric values
# The non-quantitative (categorical) columns will have NaN at new timestamps
df_weather_halfhour = df_weather_hourly.resample('30T').asfreq()
df_weather_halfhour[quantitative_cols] = df_weather_halfhour[quantitative_cols].interpolate(method='time')
df_weather_halfhour[quantitative_cols] = df_weather_halfhour[quantitative_cols].round(2)
df_weather_halfhour.head()

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity
time,,,,,,,,,,
2011-11-01 00:00:00,13.63,160.0,13.49,11.48,1008.14,13.49,3.11,rain,clear-night,0.88
2011-11-01 00:30:00,13.44,157.0,13.11,11.53,1008.01,13.11,3.10,NaN,NaN,0.90
2011-11-01 01:00:00,13.26,154.0,12.73,11.58,1007.88,12.73,3.08,rain,partly-cloudy-night,0.93
2011-11-01 01:30:00,13.10,157.5,13.19,11.86,1007.48,13.19,3.40,NaN,NaN,0.92
2011-11-01 02:00:00,12.94,161.0,13.65,12.14,1007.09,13.65,3.71,rain,clear-night,0.91


In [134]:
# Identify qualitative columns (categorical dtype)
qualitative_cols = df_weather_halfhour.select_dtypes(include=['category', 'object']).columns.tolist()

# Forward-fill to copy previous values at new timestamps

df_weather_halfhour[qualitative_cols] = df_weather_halfhour[qualitative_cols].ffill()

df_weather_halfhour.head()

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity
time,,,,,,,,,,
2011-11-01 00:00:00,13.63,160.0,13.49,11.48,1008.14,13.49,3.11,rain,clear-night,0.88
2011-11-01 00:30:00,13.44,157.0,13.11,11.53,1008.01,13.11,3.10,rain,clear-night,0.90
2011-11-01 01:00:00,13.26,154.0,12.73,11.58,1007.88,12.73,3.08,rain,partly-cloudy-night,0.93
2011-11-01 01:30:00,13.10,157.5,13.19,11.86,1007.48,13.19,3.40,rain,partly-cloudy-night,0.92
2011-11-01 02:00:00,12.94,161.0,13.65,12.14,1007.09,13.65,3.71,rain,clear-night,0.91


In [136]:
df_weather_halfhour.to_parquet('data/01_interim/weather_hourly_darksky_cleaned.parquet')

In [122]:
# Calculate the mean of quantitative variables for each hour of the day
#df_weather_hourly['hour'] = df_weather_hourly['time'].dt.hour
#hourly_means = df_weather_hourly.groupby('hour')[quantitative_cols].mean()
#hourly_means

In [123]:
#hourly_means.plot(subplots=True, layout=(4, 2), figsize=(14, 16), title=[f"{col} vs Hour of Day" for col in hourly_means.columns])
#plt.tight_layout()
#plt.show()

# Consumption